<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller # 9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

#### Librerías

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\user\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


#### Cargar datos

In [2]:
path = r'C:/Users/user/Documents/ESPECIALIZACION/LNP/Archivos/reviews_vidjew_es.csv'
data = pd.read_csv(path)
print(f"Tenemos {data.shape[0]} registros.")
data.sample(5)

Tenemos 1000 registros.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
145,es_0248447,product_es_0859850,reviewer_es_0742996,2,"Es justito de calidad, lo devolví",No me gustó,es,jewelry
992,es_0394384,product_es_0023876,reviewer_es_0440817,1,La verdad lo he devuelto pues cuando lo compré...,Muy malo,es,jewelry
313,es_0647940,product_es_0990724,reviewer_es_0093474,5,"Como sirve Amazon, rapido, productos de calida...",El servicio y la rapidez,es,video_games
564,es_0564139,product_es_0983142,reviewer_es_0842147,3,"Esta bien pero no es como el minecraft normal,...",Es buen juego pero es aburrido,es,video_games
980,es_0764733,product_es_0478112,reviewer_es_0792762,3,Entretenida campaña. Al precio que está en est...,Un juego correcto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Procesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

#### Crear una represantación de los documentos en forma de diccionario

In [4]:
dictionary = Dictionary(data['Pre-Procesado'].values)

#### Filtrar palabras muy frecuentes o infrecuentes

In [5]:
dictionary.filter_extremes(no_below=5, no_above=0.5) 

#### Corpus

In [6]:
corpus = [dictionary.doc2bow(text) for text in data['Pre-Procesado'].values]

#### Entrenar el modelo

In [45]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics =5, passes=200)

In [46]:
model.print_topics(num_words=20)

[(0,
  '0.063*"juego" + 0.038*"perfecto" + 0.033*"si" + 0.023*"jugar" + 0.023*"compra" + 0.020*"ps" + 0.017*"mejor" + 0.014*"bien" + 0.012*"tiempo" + 0.012*"gracias" + 0.012*"español" + 0.012*"gran" + 0.011*"cumple" + 0.010*"funciona" + 0.010*"perfectamente" + 0.010*"producto" + 0.010*"ser" + 0.009*"viene" + 0.009*"versión" + 0.009*"mas"'),
 (1,
  '0.083*"calidad" + 0.051*"precio" + 0.042*"buena" + 0.038*"producto" + 0.032*"buen" + 0.028*"bien" + 0.017*"puede" + 0.015*"mala" + 0.015*"recomiendo" + 0.014*"mas" + 0.014*"pedir" + 0.013*"bastante" + 0.013*"llegado" + 0.012*"tiempo" + 0.011*"llego" + 0.009*"entrega" + 0.009*"cierre" + 0.009*"botones" + 0.008*"pulsera" + 0.008*"viene"'),
 (2,
  '0.032*"mando" + 0.020*"bastante" + 0.016*"cable" + 0.015*"original" + 0.015*"bien" + 0.015*"switch" + 0.014*"si" + 0.014*"día" + 0.012*"nintendo" + 0.012*"funciona" + 0.012*"puede" + 0.011*"demás" + 0.011*"pulsera" + 0.010*"perfectamente" + 0.010*"carga" + 0.009*"consola" + 0.009*"problema" + 0.009*"

###  `[25 pts]` Punto 3: Visualización de LDA

In [47]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

#### Resultados

In [48]:
d = dictionary.doc2bow(["calidad", "juego", "buena", "bonito"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics

[(1, 0.24381305),
 (2, 0.42239544),
 (3, 0.04000062),
 (4, 0.04034772),
 (5, 0.25344315)]

In [49]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [50]:
for t in range(0,5):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Procesado'].apply(lambda doc: get_doc_top_n(doc, t))

In [51]:
data

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado,topic_0,topic_1,topic_2,topic_3,topic_4
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]",0.033384,0.866577,0.033334,0.033334,0.033371
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...",0.206963,0.034238,0.389828,0.335643,0.033335
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...",0.209685,0.028594,0.704108,0.028595,0.029053
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...",0.025707,0.025299,0.394085,0.026391,0.528473
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...",0.017019,0.274447,0.396006,0.016831,0.295702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,es_0427672,product_es_0899883,reviewer_es_0474735,2,En la foto parece que la cadena sea más gruesa...,Cadena muy fina,es,jewelry,"[foto, parece, cadena, gruesa, fina, precio, p...",0.022365,0.022705,0.022871,0.022514,0.909532
996,es_0059540,product_es_0702564,reviewer_es_0754753,3,"Bien por su precio, la cadena demasiado fina.",E,es,jewelry,"[bien, precio, cadena, demasiado, fina]",0.033385,0.034630,0.033399,0.033666,0.864930
997,es_0303349,product_es_0678671,reviewer_es_0172313,5,Muy bien!! Ahora tengo organizados los pendien...,Bonito,es,jewelry,"[bien, ahora, organizados, pendientes]",0.051157,0.050594,0.051338,0.050139,0.796805
998,es_0659961,product_es_0294943,reviewer_es_0554554,4,"Las tapas para los joystics no están nada mal,...",No está mal pero sólo he probado las fundas.,es,video_games,"[tapas, joystics, mal, ayudan, montón, joystic...",0.937396,0.015415,0.015932,0.015597,0.015670


In [52]:
for t in range(0,8):
    print(f"--------------------- TEMA {t} ---------------------")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"Comentarios #{i}")
        print(articulo[:500])
        print()
    print()

--------------------- TEMA 0 ---------------------
Comentarios #0
Este es un aviso para la gente que compran de Amazon después de tantos años comprando de Amazon he tenido que devolver piezas de ordenador porque fallaba siempre después de devolver todas las piezas ahora ni me contestan ni me devuelvan el dinero es una estafa devolver productos a Amazon os lo juro llevo ya dos semanas y no mcen nada llamo y mando emails y nada son más de 2300 euros me lo han robado y los voy a denunciar os aviso para que lo sepáis si os toca devolver algo a Amazon mejor hacéis 

Comentarios #1
Quizá un poco caro por 25€ pero es que el juego es brutal. Uno de los mejores. Esta edición tiene todo el contenido disponible y funciona perfectamente en Xbox One y se instala en Español. La carátula es exactamente la que viene en la foto, la que pone Xbox One y Xbox 360. Pero OJO, es versión NTSC, si lo compras para tu Xbox 360 PAL no te funcionará

Comentarios #2
Engaño ,me han eviado una versión que no es uk o

###  `[3 pts]` Preguntas

***¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?****

Lo que puedo concluir es que cada modelo tiene sus ventajas y desventajas y se debe analizar según el dataset y lo que se busca obtener cual sería el mejor modelo para ser desarrollado, ya que a pesar de ser los mismos datos en los tres talleres, todos tuvieron salidas de información diferentes al igual que sus resultados.

#### Elaborado por: Sandra Karina Osorio Ch. G51